In [180]:
'''
a derniere étape de notre projet est d'utiliser les modèles de Machine Learning choisi pour essayer de prédire 
les données de 2022 puis de comparer les écarts avec le reel
'''
#ETAPE 1 : Pré_processing and feature engineering de la base de donnée 2022 pour la rendre exploitable

#On importe le module et le df
import pandas as pd
df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

#On ne garde que les lignes 2022
df_2022=df_2022_full[(df_2022_full['year']==2022)]

#On ajoute les continents
df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

#On supprime les lignes contenant des vides
df_2022=df_2022.dropna()

#On supprime la colonne année
df_2022=df_2022.drop(['year'],axis=1)

#On sépare les variables explicatives de la variable cible et on enleve les country name inutile à la modélisation
X_test2022=df_2022.drop(['Life Ladder','Country name'],axis=1)
y_test2022=df_2022['Life Ladder']

#On encode les continents de X_2022
X_test2022=pd.get_dummies(X_test2022,columns=['Regional indicator'])

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_test2022[cols]=scaler.fit_transform(X_test2022[cols])

#X_test2022 et y_test2022 sont alors respectivements nos variables explicatives et notre variable cible de notre jeu de test


In [181]:
#ETAPE 2 : Lors de notre phase d'exploration des modèles de machine learning nous avions nos jeux Train et Test de toutes
#les années reunis. Nous allons concatener ces jeux dans le but d'avoir notre jeu d'entrainement 

#On importe les jeux de données
X_train=pd.read_csv('X_train.csv')
X_test=pd.read_csv('X_test.csv')
y_train=pd.read_csv('y_train.csv')
y_test=pd.read_csv('y_test.csv')

#On concatene les jeux de données
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

#Pour le mettre sous forme d'array
y_train2021=y_train2021['Life Ladder'].values

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_train2021[cols]=scaler.fit_transform(X_train2021[cols])


In [182]:
#EtAPE 3: Faire tourner le modele de RANDOM FOREST pour la regression et le tester

#Nous avons donc X_train2021,X_test2022,y_train2021,y_test2022
#Comme jeu d'entrainement et jeu de test

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=20,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022,y_test2022))


Score sur jeu d'entrainement 2021 : 0.9855635317982022
Score sur jeu de test 2022 : 0.7955268471437675


In [183]:
#On fait une prédiction du Life Ladder de 2022 par pays grâce a notre modèle random forest.
y_predic2022 = rf_model.predict(X_test2022)

#On arrondi cette prédictions à 2 décimals
import numpy as np
y_predic2022=np.round(y_predic2022,2)

#On ajoute a notre df_2022 une colonne avec notre prédiction
df_2022['Predictions']=y_predic2022
df_2022=df_2022.drop(cols,axis=1)

#On ajoute une colonne de variations en %
df_2022['Ecart en %']=np.round((df_2022['Predictions']-df_2022['Life Ladder'])/df_2022['Life Ladder']*100,1)


In [184]:
df_2022.head(10)

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.98,-4.4
2,Argentina,Latin America and Caribbean,6.26,6.19,-1.1
3,Armenia,Commonwealth of Independent States,5.38,4.93,-8.4
4,Australia,North America and ANZ,7.04,6.97,-1.0
5,Austria,Western Europe,7.00,6.74,-3.7
6,Bangladesh,South Asia,3.41,4.08,19.6
7,Belgium,Western Europe,6.86,6.97,1.6
8,Benin,Sub-Saharan Africa,4.22,3.92,-7.1
9,Bolivia,Latin America and Caribbean,5.93,5.24,-11.6
10,Botswana,Sub-Saharan Africa,3.44,4.57,32.8


In [185]:
df_2022.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

#Grâce à ce df on s'aperçoit que notre modele prédit plutot bien, il a un peu plus de mal a prédire les valeurs extremes


,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.761111,-7.083333
Southeast Asia,5.530000,5.105000,-6.925000
Commonwealth of Independent States,5.592500,5.198750,-6.800000
East Asia,5.973333,5.600000,-6.400000
Central and Eastern Europe,6.044545,5.745455,-4.590909
North America and ANZ,6.907500,6.792500,-1.725000
Western Europe,6.893333,6.790556,-1.538889
South Asia,4.270000,4.176667,1.066667
Sub-Saharan Africa,4.289643,4.259643,1.750000


In [192]:
display(df_2022.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022.sort_values('Life Ladder',ascending=True).tail(10))

#On pourrait se demander si lorsqu'il ya des valeurs un peu plus extremes, le continent n'entraine pas le modele vers 
#une réponse plus cohérente en fonction du continents. J'ai donc essayé le même modèle sans la variable continent
#En effet cela donne des meilleurs resultats sur les valeurs extrêmes mais globalement cela donne des résultat moins précis
#sur le reste

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.01,70.6
93,Sierra Leone,Sub-Saharan Africa,2.56,3.53,37.9
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.03,25.5
113,Zimbabwe,Sub-Saharan Africa,3.30,4.10,24.2
66,Malawi,Sub-Saharan Africa,3.36,3.77,12.2
6,Bangladesh,South Asia,3.41,4.08,19.6
10,Botswana,Sub-Saharan Africa,3.44,4.57,32.8
19,Comoros,Sub-Saharan Africa,3.55,3.81,7.3
101,Tanzania,Sub-Saharan Africa,3.62,4.10,13.3
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.93,-15.8
22,Costa Rica,Latin America and Caribbean,7.08,6.65,-6.1
64,Luxembourg,Western Europe,7.23,6.91,-4.4
83,Norway,Western Europe,7.30,7.42,1.6
78,Netherlands,Western Europe,7.39,7.04,-4.7
98,Sweden,Western Europe,7.43,7.46,0.4
45,Iceland,Western Europe,7.45,7.41,-0.5
26,Denmark,Western Europe,7.55,7.40,-2.0
50,Israel,Middle East and North Africa,7.66,6.79,-11.4
33,Finland,Western Europe,7.73,7.51,-2.8
